In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.svm import SVR
import joblib
from sklearn.multioutput import RegressorChain
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import KFold
from sklearn.linear_model import ElasticNet
import tensorflow.keras as keras
import tensorflow as tf

In [2]:
path = "D:/LG_Radar"
train = pd.read_csv(path+"/train.csv")
test = pd.read_csv(path+"/test.csv")
submission = pd.read_csv(path+"/sample_submission.csv")

In [9]:
def lg_nrmse(gt, preds):
    # 각 Y Feature별 NRMSE 총합
    # Y_01 ~ Y_08 까지 20% 가중치 부여
    all_nrmse = []
    for idx in range(14): # ignore 'ID'
        rmse = mean_squared_error(gt[:,idx], preds[:,idx], squared=False)
        nrmse = rmse/np.mean(np.abs(gt[:,idx]))
        all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse[:8]) + 1.0 * np.sum(all_nrmse[8:14])
#     print("Test score: "all_nrmse)
    return score

In [10]:
train

,ID,X_01,X_02,X_03,X_04,X_05,X_06,X_07,X_08,X_09,...,Y_05,Y_06,Y_07,Y_08,Y_09,Y_10,Y_11,Y_12,Y_13,Y_14
0,TRAIN_00001,70.544,103.320,67.47,1,101.892,74.983,29.45,62.38,245.71,...,29.632,16.083,4.276,-25.381,-25.529,-22.769,23.792,-25.470,-25.409,-25.304
1,TRAIN_00002,69.524,103.321,65.17,1,101.944,72.943,28.73,61.23,233.61,...,33.179,16.736,3.229,-26.619,-26.523,-22.574,24.691,-26.253,-26.497,-26.438
2,TRAIN_00003,72.583,103.320,64.07,1,103.153,72.943,28.81,105.77,272.20,...,31.801,17.080,2.839,-26.238,-26.216,-22.169,24.649,-26.285,-26.215,-26.370
3,TRAIN_00004,71.563,103.320,67.57,1,101.971,77.022,28.92,115.21,255.36,...,34.503,17.143,3.144,-25.426,-25.079,-21.765,24.913,-25.254,-25.021,-25.345
4,TRAIN_00005,69.524,103.320,63.57,1,101.981,70.904,29.68,103.38,241.46,...,32.602,17.569,3.138,-25.376,-25.242,-21.072,25.299,-25.072,-25.195,-24.974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39602,TRAIN_39603,66.465,103.320,62.27,1,103.150,66.825,30.20,77.83,298.05,...,29.194,16.582,3.410,-26.486,-26.581,-22.772,24.261,-26.491,-26.584,-26.580
39603,TRAIN_39604,66.465,103.321,62.77,1,102.021,66.825,29.21,102.25,270.67,...,29.859,15.659,3.406,-27.308,-27.203,-24.674,23.427,-27.250,-27.334,-27.325
39604,TRAIN_39605,68.504,103.320,64.67,1,103.144,68.864,29.96,102.61,198.07,...,24.720,16.823,3.215,-26.502,-26.687,-22.577,24.301,-26.388,-26.425,-26.601
39605,TRAIN_39606,66.465,103.320,63.67,1,102.025,67.845,30.30,112.60,275.52,...,26.412,15.757,4.216,-26.760,-26.634,-24.066,23.305,-26.536,-26.751,-26.635


In [11]:
train.columns

Index(['ID', 'X_01', 'X_02', 'X_03', 'X_04', 'X_05', 'X_06', 'X_07', 'X_08',
       'X_09', 'X_10', 'X_11', 'X_12', 'X_13', 'X_14', 'X_15', 'X_16', 'X_17',
       'X_18', 'X_19', 'X_20', 'X_21', 'X_22', 'X_23', 'X_24', 'X_25', 'X_26',
       'X_27', 'X_28', 'X_29', 'X_30', 'X_31', 'X_32', 'X_33', 'X_34', 'X_35',
       'X_36', 'X_37', 'X_38', 'X_39', 'X_40', 'X_41', 'X_42', 'X_43', 'X_44',
       'X_45', 'X_46', 'X_47', 'X_48', 'X_49', 'X_50', 'X_51', 'X_52', 'X_53',
       'X_54', 'X_55', 'X_56', 'Y_01', 'Y_02', 'Y_03', 'Y_04', 'Y_05', 'Y_06',
       'Y_07', 'Y_08', 'Y_09', 'Y_10', 'Y_11', 'Y_12', 'Y_13', 'Y_14'],
      dtype='object')

In [12]:
X = np.array(train[list(train.columns[1:57])])
y = np.array(train[list(train.columns[57:])])

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, np.array(y), random_state=42)

In [14]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled, X_test_scaled = scaler.transform(X_train), scaler.transform(X_test) 

# Make base models

각각의 모델의 에러를 가중치로 평균내어 예측한다.

In [18]:
for i in range(5):
    models_list=[]
    rf_model = RandomForestRegressor()
    rf_model.fit(X_train, y_train)
    print("Fold",i+1,":",lg_nrmse(y_train, rf_model.predict(X_train)))
    print("Fold",i+1,":",lg_nrmse(y_test, rf_model.predict(X_test)))

Fold 1 : 0.7363600828828816
Fold 1 : 1.9741007480163444
Fold 2 : 0.7350314604471373
Fold 2 : 1.9735237739728189
Fold 3 : 0.7366224092684309
Fold 3 : 1.9727716031946225
Fold 4 : 0.7345493019945082
Fold 4 : 1.9763436957693863


KeyboardInterrupt: 

In [10]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled, X_test_scaled = scaler.transform(X_train), scaler.transform(X_test) 
svm_model = MultiOutputRegressor(SVR())
svm_model.fit(X_train_scaled, y_train)
print(lg_nrmse(y_test, svm_model.predict(X_test_scaled)))

In [25]:
xgb_model = MultiOutputRegressor(XGBRegressor(max_depth=5, learning_rate=0.1, alpha=4, gamma=1, reg_lambda=7))
xgb_model.fit(X_train, y_train)
print(lg_nrmse(y_train, xgb_model.predict(X_train)))
print(lg_nrmse(y_test, xgb_model.predict(X_test)))

[0.2523191488237303, 0.3471716072995733, 0.34291181589665465, 0.17924742818676148, 0.07528309089874936, 0.0861067120607161, 0.12551482815256845, 0.023022719935640458, 0.022867724240378702, 0.03635485902932705, 0.03195408792574978, 0.022998638186077818, 0.022940338792546144, 0.02299854675715869]
1.878007016436511
[0.2561240472193856, 0.35727586271824735, 0.3479654264452222, 0.18822887493532348, 0.08036300316087812, 0.11843145716290114, 0.12995804292448906, 0.023770709108455572, 0.02353078587312592, 0.03925235171604544, 0.033328623048510524, 0.02364746084163248, 0.023614672274852578, 0.02370478280264867]
1.9696195849666984


# Make k-fold model list

## Random Forest 

In [19]:
def prediction(models):
    pred=[]
    for i in range(5):
        pred.append(models[i].predict(X_test))
    return pred

In [20]:
def prediction_scaled(models):
    pred=[]
    for i in range(5):
        pred.append(models[i].predict(X_test_scaled))
    return pred

In [21]:
kf = KFold(n_splits=5)
rf_list=[]
for train_index, test_index in kf.split(X_train):
    X_train_, X_test_ = X_train[train_index], X_train[test_index]
    y_train_, y_test_ = y_train[train_index], y_train[test_index]
    rf_model = RandomForestRegressor()
    rf_model.fit(X_train_, y_train_)
    print("Train: ", lg_nrmse(y_train_, rf_model.predict(X_train_)))
    print("Test: ",lg_nrmse(y_test_, rf_model.predict(X_test_)))
    rf_list.append(rf_model)

Train:  0.7373437847478075
Test:  1.9710866741217392
Train:  0.7373979350284185
Test:  1.9570843673495357
Train:  0.7357111271048333
Test:  1.9943886309535788
Train:  0.7367797694270555
Test:  1.960170106059873
Train:  0.7381142407559815
Test:  1.9596865943689303


In [22]:
pred_1 = prediction(rf_list)
pred_1 = np.sum(pred_1, axis=0)/5

In [23]:
lg_nrmse(y_test, pred_1)

1.9665793951424426

## XGBoost 

In [24]:
X.shape

(39607, 56)

In [25]:
kf = KFold(n_splits=5)
xgb_list=[]

for train_index, test_index in kf.split(X_train):
    X_train_, X_test_ = X_train[train_index], X_train[test_index]
    y_train_, y_test_ = y_train[train_index], y_train[test_index]
    xgb_model = MultiOutputRegressor(XGBRegressor(max_depth=5, learning_rate=0.1, alpha=3, gamma=1, reg_lambda=5)) 
    xgb_model.fit(X_train_, y_train_)
    print("Train: ", lg_nrmse(y_train_, xgb_model.predict(X_train_)))
    print("Test: ",lg_nrmse(y_test_, xgb_model.predict(X_test_)))
    xgb_list.append(xgb_model)

Train:  1.8563585295090541
Test:  1.9617683714120815
Train:  1.8596502580152299
Test:  1.944956216047597
Train:  1.8502479302724155
Test:  1.9923242327215793
Train:  1.8540785987613204
Test:  1.9570433303272305
Train:  1.8642639861824906
Test:  1.953269265013851


In [26]:
pred_2 = prediction(xgb_list)
pred_2 = np.sum(pred_2, axis=0)/5

In [27]:
lg_nrmse(y_test, pred_2)

1.962204435826028

## Linear model 

In [28]:
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import BayesianRidge

In [29]:
kf = KFold(n_splits=5)
linear_list=[]

for train_index, test_index in kf.split(X_train_scaled):
    X_train_, X_test_ = X_train_scaled[train_index], X_train_scaled[test_index]
    y_train_, y_test_ = y_train[train_index], y_train[test_index]
    linear_model = MultiOutputRegressor(HuberRegressor(epsilon=1.5)) 
    linear_model.fit(X_train_, y_train_)
    print("Train: ", lg_nrmse(y_train_, linear_model.predict(X_train_)))
    print("Test: ",lg_nrmse(y_test_, linear_model.predict(X_test_)))
    linear_list.append(linear_model)

Train:  1.9827589192345323
Test:  1.9918941253622076
Train:  1.9890078595583862
Test:  1.9644218867096175
Train:  1.9749839646667762
Test:  2.0218657041761436
Train:  1.9853533877898764
Test:  1.9805697494876129
Train:  1.986314359267524
Test:  1.9779123677731476


In [30]:
pred_3 = prediction_scaled(linear_list)
pred_3 = np.sum(pred_3, axis=0)/5

In [31]:
lg_nrmse(y_test, pred_3)

1.9935594147026159

## ANN 

In [32]:
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor = "val_loss", factor=0.5, patience=3, verbose=0)
es = keras.callbacks.EarlyStopping(monitor = "val_loss", patience=10)
mc = keras.callbacks.ModelCheckpoint(monitor = "val_loss", verbose=3, save_best_only=True,
                                     mode="min", filepath="./checkpoint.h5")

In [33]:
kf = KFold(n_splits=5)
model_list=[]
for train_index, test_index in kf.split(X_train_scaled):
    X_train, X_test = X_train_scaled[train_index], X_train_scaled[test_index]
    y_train_, y_test_ = y_train[train_index], y_train[test_index]

    Input = keras.layers.Input(shape=X_train.shape[1])
    x = keras.layers.Dense(32, activation="relu", kernel_initializer=keras.initializers.he_normal)(Input)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dense(32, activation="relu", kernel_initializer=keras.initializers.he_normal)(x)
    x = keras.layers.BatchNormalization()(x)

    Output = keras.layers.Dense(14, activation="linear", kernel_initializer=keras.initializers.he_normal)(x)


    model = keras.models.Model(inputs = Input, outputs = Output)
    model.compile(loss="mse", optimizer=keras.optimizers.Adam(learning_rate=0.01), metrics=["mse", "mae"])
    model.fit(X_train, y_train_, epochs=70, validation_data = (X_test, y_test_), batch_size=128, callbacks=[reduce_lr, es], verbose=0)
    print(lg_nrmse(y_train_, model.predict(X_train)))
    print(lg_nrmse(y_test_, model.predict(X_test)))    
    model_list.append(model)

1.9624414790802693
1.9886572393559687
1.96888902440219
1.9741928866137286
1.9533629444112042
2.0194261307426085
1.9689037758981776
1.9887502938584043
1.9662636179000432
1.972804061275206


In [34]:
pred_4 = prediction_scaled(model_list)
pred_4 = np.sum(pred_4, axis=0)/5

In [48]:
pd.DataFrame(pred_svm)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1.409819,1.252312,1.163701,13.512946,31.008136,16.635279,3.299466,-26.310833,-26.330778,-22.271583,24.359146,-26.188435,-26.190361,-26.306680
1,1.415609,1.250920,1.147291,12.757977,31.704951,16.662048,3.183028,-26.128144,-26.183824,-22.251028,24.489667,-26.072201,-26.078579,-26.063945
2,1.564963,1.100217,1.134266,14.258767,31.306923,16.866931,3.428086,-25.977658,-26.054103,-22.115169,24.587552,-25.961441,-26.002260,-25.966161
3,1.668950,1.354370,1.267215,15.352608,32.625482,17.454612,3.025557,-25.503381,-25.574858,-21.553498,25.203769,-25.508678,-25.486801,-25.516549
4,1.286316,0.939406,0.803248,15.372050,32.355104,17.091091,3.059540,-25.535459,-25.482908,-21.988857,24.910916,-25.368966,-25.368402,-25.472581
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39603,1.253778,0.802493,1.018040,13.490981,30.958652,16.737182,3.480894,-26.443106,-26.416641,-22.642595,24.415092,-26.296327,-26.288961,-26.306986
39604,1.183491,0.719369,0.934206,12.902357,31.205385,16.707389,3.415066,-26.545678,-26.556356,-23.007535,24.359420,-26.463875,-26.521575,-26.559217
39605,1.209365,0.909526,1.004215,13.393399,31.922727,16.778812,3.173302,-26.346610,-26.440189,-22.660900,24.503594,-26.398515,-26.415295,-26.454039
39606,1.078828,0.797547,0.853858,13.073846,31.515225,16.854181,3.326421,-26.457356,-26.537674,-22.713752,24.508952,-26.481735,-26.422933,-26.510171


In [52]:
print("RF error: ",lg_nrmse(y_test, pred_1))
print("XGBoost error: ",lg_nrmse(y_test, pred_2))
print("Linear error: ",lg_nrmse(y_test, pred_3))
print("ANN error: ",lg_nrmse(y_test, pred_4))

RF error:  1.9665793951424426
XGBoost error:  1.962204435826028
Linear error:  1.9935594147026159
ANN error:  1.9820293840430665


In [217]:
lg_nrmse(y_test, pred_4)

[0.2589138539962884, 0.3583680360603153, 0.3510731064799994, 0.19966994623390213, 0.07987238862756645, 0.10961651991275326, 0.1304997282588233, 0.024281577608257002, 0.024144656736569663, 0.03892045253052741, 0.03354363242586301, 0.024324059440830454, 0.02418459248861957, 0.02417606413973205]


1.9840476463756285

In [61]:
print(lg_nrmse(y_test, pred_1*0.3 + pred_2*0.3 + pred_3*0.2+ pred_4*0.2 ))
print(lg_nrmse(y_test, pred_1*0.4 + pred_2*0.4 + pred_3*0.1+ pred_4*0.1 ))
print(lg_nrmse(y_test, pred_1*0.5 + pred_2*0.5))

1.9642407195732519
1.96151414843748
1.9603195716142015


In [62]:
pd.DataFrame(pred_1).describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
count,9902.000000,9902.000000,9902.000000,9902.000000,9902.000000,9902.000000,9902.000000,9902.000000,9902.000000,9902.000000,9902.000000,9902.000000,9902.000000,9902.000000
mean,1.349101,1.053956,1.012755,13.571874,31.228986,16.481148,3.159973,-26.309497,-26.323144,-22.425019,24.300871,-26.252422,-26.247961,-26.259937
std,0.059896,0.063055,0.050308,0.626021,0.466314,0.531225,0.075595,0.184256,0.176406,0.274051,0.155445,0.178616,0.179105,0.177642
min,0.812870,0.608130,0.612986,11.362686,29.289798,2.051434,2.352288,-27.080992,-27.082126,-25.496062,23.340508,-27.011742,-27.023388,-27.029548
25%,1.317052,1.021950,0.984384,13.158926,30.934046,16.455461,3.116962,-26.432678,-26.439252,-22.531464,24.211503,-26.368801,-26.365687,-26.376172
50%,1.355127,1.064123,1.018438,13.516398,31.235549,16.556074,3.167897,-26.306546,-26.322865,-22.362247,24.297738,-26.251016,-26.246094,-26.258504
75%,1.389600,1.096312,1.047342,13.898447,31.527239,16.642084,3.209987,-26.200307,-26.222172,-22.254417,24.383009,-26.149881,-26.144709,-26.158477
max,1.534856,1.243144,1.153014,16.548558,33.108836,17.425948,3.417972,-25.482920,-25.499632,-21.546024,25.134070,-25.430376,-25.425784,-25.422922


In [63]:
pd.DataFrame(pred_2).describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
count,9902.000000,9902.000000,9902.000000,9902.000000,9902.000000,9902.000000,9902.000000,9902.000000,9902.000000,9902.000000,9902.000000,9902.000000,9902.000000,9902.000000
mean,1.351346,1.055491,1.011978,13.618754,31.286003,16.528347,3.155011,-26.290165,-26.304111,-22.398470,24.324184,-26.234230,-26.229206,-26.241220
std,0.071950,0.077015,0.062419,0.599252,0.431342,0.541811,0.078213,0.185108,0.176240,0.285738,0.143130,0.178458,0.179770,0.177486
min,1.033146,0.747062,0.736616,10.492521,28.337490,-0.717723,2.733603,-26.814575,-26.806870,-23.993128,22.743372,-26.760487,-26.754108,-26.785782
25%,1.306085,1.012376,0.974181,13.221472,31.030931,16.510699,3.106500,-26.416183,-26.421588,-22.496120,24.235567,-26.352805,-26.349106,-26.359123
50%,1.358518,1.065879,1.014653,13.553483,31.301488,16.588615,3.163809,-26.276057,-26.295508,-22.327577,24.318858,-26.223493,-26.219795,-26.231847
75%,1.403412,1.109458,1.054923,13.933627,31.554007,16.664490,3.208207,-26.176314,-26.199567,-22.219856,24.402910,-26.127316,-26.120625,-26.134768
max,1.530320,1.255145,1.196381,16.222111,33.176350,17.552164,3.408603,-25.390667,-25.398842,-21.403082,25.038754,-25.340221,-25.384708,-25.332043


In [64]:
pd.DataFrame(pred_3).describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
count,9902.000000,9902.000000,9902.000000,9902.000000,9902.000000,9902.000000,9902.000000,9902.000000,9902.000000,9902.000000,9902.000000,9902.000000,9902.000000,9902.000000
mean,1.347072,1.042691,0.999473,13.731768,31.496827,16.661681,3.144959,-26.260510,-26.274511,-22.331538,24.382779,-26.203387,-26.198858,-26.211876
std,0.065651,0.074955,0.064218,0.550516,0.410554,0.109416,0.074596,0.165446,0.155994,0.231887,0.118669,0.157614,0.158932,0.157083
min,0.884697,0.550843,0.620074,8.124358,27.148525,15.700822,2.561409,-28.048545,-28.054678,-23.636968,23.350339,-27.942365,-27.941429,-27.929147
25%,1.303133,0.993821,0.955547,13.393486,31.243543,16.591957,3.095165,-26.386161,-26.390323,-22.484966,24.306158,-26.320600,-26.317239,-26.327231
50%,1.345638,1.047165,0.997059,13.697098,31.503615,16.660047,3.145661,-26.246865,-26.264455,-22.302048,24.381327,-26.191883,-26.185758,-26.201434
75%,1.390702,1.093675,1.043108,14.028449,31.764068,16.732168,3.194454,-26.142277,-26.166613,-22.167623,24.458678,-26.093899,-26.087601,-26.102884
max,1.610381,1.278409,1.262032,19.892281,34.468162,17.480525,3.666826,-25.059762,-25.227780,-21.269397,25.216537,-25.048424,-25.055759,-25.031818


In [65]:
pd.DataFrame(pred_4).describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
count,9902.000000,9902.000000,9902.000000,9902.000000,9902.000000,9902.000000,9902.000000,9902.000000,9902.000000,9902.000000,9902.000000,9902.000000,9902.000000,9902.000000
mean,1.349894,1.060743,1.014432,13.644048,31.287966,16.532440,3.155176,-26.299282,-26.313416,-22.398985,24.331169,-26.240171,-26.236973,-26.247469
std,0.058928,0.063733,0.047547,0.729952,0.581669,0.356090,0.082015,0.197379,0.190259,0.282319,0.177603,0.192726,0.191049,0.189304
min,1.054308,0.788719,0.732965,10.913054,28.748363,12.403337,2.826491,-27.314505,-27.254833,-23.688684,23.479712,-27.252344,-27.368256,-27.077139
25%,1.317067,1.028653,0.988696,13.170562,30.912048,16.417405,3.104764,-26.427926,-26.442342,-22.545870,24.214209,-26.365055,-26.365636,-26.372887
50%,1.356618,1.072330,1.018994,13.617737,31.305668,16.576830,3.157536,-26.286353,-26.307411,-22.352126,24.329472,-26.230083,-26.225998,-26.241491
75%,1.389458,1.103584,1.046019,14.079959,31.668103,16.722656,3.208437,-26.181649,-26.197401,-22.208593,24.440190,-26.124598,-26.119041,-26.134174
max,1.539703,1.335669,1.227690,17.776194,33.340080,18.222858,3.454175,-24.129368,-24.841379,-21.234125,25.431204,-24.316940,-24.944195,-24.835608


In [73]:
pd.DataFrame(svm_pred).describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
count,39608.000000,39608.000000,39608.000000,39608.000000,39608.000000,39608.000000,39608.000000,39608.000000,39608.000000,39608.000000,39608.000000,39608.000000,39608.000000,39608.000000
mean,1.349145,1.042576,0.999886,13.807512,31.662310,16.686632,3.142152,-26.253760,-26.268223,-22.313334,24.410127,-26.195770,-26.191585,-26.204262
std,0.122910,0.134006,0.123466,0.778485,0.603484,0.229806,0.142903,0.240496,0.233475,0.337457,0.241237,0.234945,0.236529,0.234818
min,0.830097,0.470127,0.462327,10.812225,28.984126,15.561853,2.515478,-27.144490,-27.106776,-23.841441,23.424576,-27.047160,-27.033485,-27.075540
25%,1.268148,0.954372,0.918142,13.285210,31.266037,16.534222,3.048303,-26.412139,-26.421039,-22.487552,24.253113,-26.348443,-26.347380,-26.357605
50%,1.350986,1.044732,1.001222,13.774401,31.671596,16.680582,3.142173,-26.255218,-26.271112,-22.268132,24.403569,-26.197576,-26.194191,-26.206563
75%,1.431768,1.130871,1.081794,14.280427,32.068018,16.829953,3.235456,-26.102717,-26.124073,-22.090990,24.559343,-26.050182,-26.045275,-26.057692
max,1.941234,1.564941,1.539777,17.548961,34.285931,17.908373,3.856927,-24.959593,-24.980905,-21.082241,25.684221,-24.950451,-24.981116,-24.944131


## SVM 

In [8]:
# kf = KFold(n_splits=5)
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)
# model_list=[]
# for train_index, test_index in kf.split(X_scaled):
#     print("TRAIN:", train_index, "TEST:", test_index)
#     X_train_, X_test_ = X_scaled[train_index], X_scaled[test_index]
#     y_train_, y_test_ = y[train_index], y[test_index]
#     svm_model = MultiOutputRegressor(SVR())
#     svm_model.fit(X_train_, y_train_)
# #     print(lg_nrmse(y_train_, svm_model.predict(X_train_)))
#     print(lg_nrmse(y_test_, svm_model.predict(X_test_)))
#     model_list.append(svm_model)

# Make model for submission

In [258]:
import joblib
joblib.dump(rf_list, "./re_list.pkl")


KeyboardInterrupt



In [252]:
test = test[list(test.columns[1:])]

In [253]:
test = test[list(test.columns[1:])]

def prediction(models):
    pred=[]
    for i in range(5):
        pred.append(models[i].predict(test))
    return pred

In [67]:
kf = KFold(n_splits=5)
xgb_list=[]

for train_index, test_index in kf.split(X):
    X_train_, X_test_ = X[train_index], X[test_index]
    y_train_, y_test_ = y[train_index], y[test_index]
    xgb_model = MultiOutputRegressor(XGBRegressor(max_depth=5, learning_rate=0.1, alpha=3, gamma=1, reg_lambda=5)) 
    xgb_model.fit(X_train_, y_train_)
    print("Train: ", lg_nrmse(y_train_, xgb_model.predict(X_train_)))
    print("Test: ",lg_nrmse(y_test_, xgb_model.predict(X_test_)))
    xgb_list.append(xgb_model)

Train:  1.8362599098162917
Test:  2.1096257994754146
Train:  1.8613118613272086
Test:  1.9971787649190031
Train:  1.8848389184293453
Test:  1.8627259476937224
Train:  1.8707999879261472
Test:  1.93782234903024
Train:  1.8925545879798196
Test:  1.8356726270260728


In [68]:
xgb_pred = prediction(xgb_list)
xgb_pred = np.sum(xgb_pred, axis=0)/5

In [76]:
svm_pred = joblib.load("./svm_pred_test.pkl")
svm_pred = np.sum(svm_pred, axis=0)/5

In [77]:
pd.DataFrame(svm_pred).describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
count,39608.000000,39608.000000,39608.000000,39608.000000,39608.000000,39608.000000,39608.000000,39608.000000,39608.000000,39608.000000,39608.000000,39608.000000,39608.000000,39608.000000
mean,1.349145,1.042576,0.999886,13.807512,31.662310,16.686632,3.142152,-26.253760,-26.268223,-22.313334,24.410127,-26.195770,-26.191585,-26.204262
std,0.122910,0.134006,0.123466,0.778485,0.603484,0.229806,0.142903,0.240496,0.233475,0.337457,0.241237,0.234945,0.236529,0.234818
min,0.830097,0.470127,0.462327,10.812225,28.984126,15.561853,2.515478,-27.144490,-27.106776,-23.841441,23.424576,-27.047160,-27.033485,-27.075540
25%,1.268148,0.954372,0.918142,13.285210,31.266037,16.534222,3.048303,-26.412139,-26.421039,-22.487552,24.253113,-26.348443,-26.347380,-26.357605
50%,1.350986,1.044732,1.001222,13.774401,31.671596,16.680582,3.142173,-26.255218,-26.271112,-22.268132,24.403569,-26.197576,-26.194191,-26.206563
75%,1.431768,1.130871,1.081794,14.280427,32.068018,16.829953,3.235456,-26.102717,-26.124073,-22.090990,24.559343,-26.050182,-26.045275,-26.057692
max,1.941234,1.564941,1.539777,17.548961,34.285931,17.908373,3.856927,-24.959593,-24.980905,-21.082241,25.684221,-24.950451,-24.981116,-24.944131


In [78]:
pd.DataFrame(xgb_pred)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1.253992,1.003732,0.984045,7.268392,25.808044,5.895844,3.004417,-24.920250,-24.953417,-20.989700,22.511444,-25.071537,-24.988447,-24.777843
1,1.243626,0.998344,0.976844,7.700372,25.801416,6.447156,3.072688,-24.969294,-25.078884,-20.971079,22.619654,-25.148291,-25.052710,-24.797722
2,1.243626,0.998344,0.976844,7.700372,25.801416,6.447156,3.072688,-24.969294,-25.078884,-20.971079,22.619654,-25.148291,-25.052710,-24.797722
3,1.243626,0.998344,0.976844,7.700372,25.801416,6.447156,3.072688,-24.969294,-25.078884,-20.971079,22.619654,-25.148291,-25.052710,-24.797722
4,1.240712,1.006653,0.962292,7.851636,25.850714,5.934018,3.016380,-24.920078,-24.970362,-20.900723,22.520702,-25.032782,-24.992897,-24.766682
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5936,1.252845,1.001154,0.981394,7.300715,25.980114,5.923337,2.940974,-24.936169,-24.961655,-20.956070,22.508844,-25.087580,-24.976589,-24.769720
5937,1.243626,0.998344,0.976844,7.700372,25.801416,6.447156,3.072688,-24.969294,-25.078884,-20.971079,22.619654,-25.148291,-25.052710,-24.797722
5938,1.243626,0.998344,0.976844,7.700372,25.801416,6.447156,3.072688,-24.969294,-25.078884,-20.971079,22.619654,-25.148291,-25.052710,-24.797722
5939,1.258557,1.013368,0.984972,7.437630,25.484470,6.083101,2.982280,-24.847860,-24.858402,-20.855862,22.505688,-24.981159,-24.897617,-24.702101


In [259]:
joblib.dump(svm_pred, "svm_submission.pkl")

['svm_submission.pkl']

In [260]:
joblib.dump(xgb_pred, "xgb_submission.pkl")

['xgb_submission.pkl']

# Submission

In [22]:
import joblib
svm_model = joblib.load("./svm.pkl")
pred_svm = joblib.load("./pred_svm.pkl")

In [12]:
X_test_submission = test[list(test.columns[1:57])]
X_test_submission_scaled = scaler.transform(X_test_submission)

In [20]:
pred_1 = rf_model.predict(X_test_submission)*0.4

NameError: name 'rf_model' is not defined

In [ ]:
pred_2 = pred_svm * 0.2

In [21]:
pred_3 = xgb_model.predict(X_test_submission)*0.4

NameError: name 'xgb_model' is not defined

In [25]:
submission_data = pred_1 + pred_3

In [28]:
submission[list(submission.columns[1:])] = submission_data

In [31]:
submission

,ID,Y_01,Y_02,Y_03,Y_04,Y_05,Y_06,Y_07,Y_08,Y_09,Y_10,Y_11,Y_12,Y_13,Y_14
0,TEST_00001,1.409515,1.210701,1.083776,13.933078,31.433623,16.756577,3.122686,-26.144023,-26.197439,-22.219550,24.426465,-26.089073,-26.061156,-26.105292
1,TEST_00002,1.461119,1.187049,1.120219,13.868451,31.126204,16.657150,3.105777,-26.138793,-26.162941,-22.235692,24.417072,-26.103904,-26.091830,-26.132909
2,TEST_00003,1.366390,1.038038,1.003808,14.027303,31.619934,16.673731,3.119544,-25.962618,-25.939933,-22.145309,24.470158,-25.925116,-25.893393,-25.882908
3,TEST_00004,1.425013,1.124561,1.045093,14.876444,32.198437,17.084132,3.110118,-25.639896,-25.654441,-21.805200,24.879766,-25.687802,-25.557868,-25.690587
4,TEST_00005,1.339584,1.039257,0.969167,14.823126,31.544785,17.015359,3.142881,-25.675690,-25.739396,-22.102114,24.768741,-25.578993,-25.634174,-25.639194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39603,TEST_39604,1.294672,0.998447,1.012693,12.803720,31.132021,16.575491,3.206131,-26.549717,-26.525038,-22.732996,24.272350,-26.438003,-26.461607,-26.466054
39604,TEST_39605,1.226949,0.896240,0.933956,14.105540,30.992607,16.591816,3.215134,-26.489779,-26.498661,-22.826320,24.361330,-26.442771,-26.478468,-26.412184
39605,TEST_39606,1.264823,0.910235,0.958471,13.050433,31.079248,16.587271,3.161601,-26.567321,-26.585213,-22.770649,24.162510,-26.500153,-26.524040,-26.507245
39606,TEST_39607,1.215846,0.927153,0.943064,12.815964,30.535386,16.598558,3.203439,-26.573993,-26.576417,-22.888135,24.211090,-26.511960,-26.530781,-26.531343


In [30]:
submission.to_csv("./ensemble_submission.csv", index=None)